In [1]:
from IPython.display import display_html


def restart_kernel():
    display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)


restart_kernel()

In [2]:
import os

# Set environment variable for TensorFlow
# os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
import tensorflow as tf

# Configure GPU memory growth
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Set memory growth before initializing GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Handle potential errors here
        print(e)

2024-04-14 04:54:04.336997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 04:54:04.503101: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 04:54:05.342017: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-04-14 04:54:05.342180: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

1 Physical GPUs, 1 Logical GPUs


2024-04-14 04:54:06.705601: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 04:54:06.982749: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14791 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0001:00:00.0, compute capability: 7.0


In [3]:
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
import cv2
import random
import os

import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image

import pandas as pd
import numpy as np
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [4]:
import numpy as np
import random
import tensorflow as tf
import torch

# Set seeds to ensure reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # For multi-GPU setups

> Preprocessing

> Initialize base model with pre-trained weights 

## Model RAW IDRID

In [5]:
import tensorflow as tf

# load the data

X_train = np.load("../../aws_s3/Idrid_224_Raw/X_train.npy")
y_train = np.load("../../aws_s3/Idrid_224_Raw/y_train.npy")
X_test = np.load("../../aws_s3/Idrid_224_Raw/X_test.npy")
y_test = np.load("../../aws_s3/Idrid_224_Raw/y_test.npy")

X_train = tf.convert_to_tensor(X_train, dtype="float32")
# y_train = tf.convert_to_tensor(y_train, dtype="float32")
X_test = tf.convert_to_tensor(X_test, dtype="float32")
# y_test = tf.convert_to_tensor(y_test, dtype="float32")

In [6]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling="avg",
)

> Create a new model on top with dropout

In [7]:
# freesze the base model
base_model.trainable = False
num_classes = 4
epochs = 50
# Define the inputs
inputs = tf.keras.Input(shape=(224, 224, 3))

# Ensure the base_model is running in inference mode.
x = base_model(inputs, training=False)

x = layers.Dropout(0.5)(x)  # Dropout layer with 50% dropout rate
# Adding FC (Fully Connected) layers
x = layers.Dense(1000)(x)
x = layers.Dropout(0.5)(x)  # Another Dropout layer with 50% dropout rate
# Adding FC (Fully Connected) layers
x = layers.Dense(500)(x)

# Adding a final layer with SoftMax activation for classification
outputs = layers.Dense(num_classes, activation="softmax")(x)

# Creating the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary(show_trainable=True)

Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 efficientnetb0 (Functional)  (None, 1280)             4049571   N          
                                                                            
 dropout (Dropout)           (None, 1280)              0         Y          
                                                                            
 dense (Dense)               (None, 1000)              1281000   Y          
                                                                            
 dropout_1 (Dropout)         (None, 1000)              0         Y          
                                                                            
 dense_1 (Dense)             (None, 500)               500500

> Train the model on new data

In [8]:
from tensorflow.keras.optimizers import SGD

# base learning rate
base_learning_rate = 1e-4
# maximum learning rate
max_learning_rate = 1e-2

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9, clipnorm=1.0)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

In [9]:
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

print("Fitting the top layer of the model")
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Fitting the top layer of the model
Epoch 1/50


2024-04-14 04:54:18.998880: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2024-04-14 04:54:19.103308: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


16/16 [==============================] - 7s 39ms/step - loss: 1.8416 - accuracy: 0.2114 - lr: 1.0000e-04
Epoch 2/50
16/16 [==============================] - 0s 26ms/step - loss: 1.6140 - accuracy: 0.2602 - lr: 0.0011
Epoch 3/50
16/16 [==============================] - 0s 26ms/step - loss: 1.6353 - accuracy: 0.3516 - lr: 0.0021
Epoch 4/50
16/16 [==============================] - 0s 26ms/step - loss: 1.4381 - accuracy: 0.3516 - lr: 0.0031
Epoch 5/50
16/16 [==============================] - 0s 26ms/step - loss: 1.3216 - accuracy: 0.4492 - lr: 0.0041
Epoch 6/50
16/16 [==============================] - 0s 26ms/step - loss: 1.4094 - accuracy: 0.4268 - lr: 0.0050
Epoch 7/50
16/16 [==============================] - 0s 26ms/step - loss: 1.2302 - accuracy: 0.4268 - lr: 0.0060
Epoch 8/50
16/16 [==============================] - 0s 26ms/step - loss: 1.2929 - accuracy: 0.4614 - lr: 0.0070
Epoch 9/50
16/16 [==============================] - 0s 26ms/step - loss: 1.1215 - accuracy: 0.5163 - lr: 0.0080

> Fine tuning: unfreeze all or part of the base model and retrain the whole model end-to-end with a very low learning rate.

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Unfreeze the base model
base_model.trainable = True
model.summary(show_trainable=True)

# Re-instantiate the optimizer
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9, clipnorm=1.0)

model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

print("Fitting the end-to-end model")
# Train end-to-end. Stop before overfit
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 efficientnetb0 (Functional)  (None, 1280)             4049571   Y          
                                                                            
 dropout (Dropout)           (None, 1280)              0         Y          
                                                                            
 dense (Dense)               (None, 1000)              1281000   Y          
                                                                            
 dropout_1 (Dropout)         (None, 1000)              0         Y          
                                                                            
 dense_1 (Dense)             (None, 500)               500500

> Evaluate on test data

In [11]:
# model.save("my_model.keras")
# 50 epochs == 41% accuracy
# 25 epochs == 45% accuracy
# 20 epochs == 45.83% accuracy
# 25 epochs == 62.5% accuracy

In [12]:
print("Test dataset evaluation")
model.evaluate(X_test, y_test)

Test dataset evaluation
1/1 [==============================] - 2s 2s/step - loss: 3.4580 - accuracy: 0.5000


[3.45804500579834, 0.5]

In [13]:
# 20 epcohs, tensor X, 0.4583
# 20 epcohs, tensor X, 0.5000
# 20 epcohs, tensor X, 0.5000
# 20 epcohs, tensor X, 0.5417
# 20 epcohs, tensor X, 0.4583
# 20 epcohs, tensor X, 0.5833
# 20 epcohs, tensor X  0.5417
# 20 epcohs, tensor X  0.5417
# 20 epcohs, tensor X  0.6667
# 20 epcohs, tensor X. 0.5000
# 20 epcohs, tensor X. 0.4583
# 20 epcohs, tensor X.  0.5000
# 20 epcohs, tensor X.  0.5417

In [14]:
# 50 epochs, tensor X, 0.5833

# Color Normalization

In [15]:
# load the data
X_train = np.load("../../aws_s3/Idrid_224_CN/X_train.npy")
y_train = np.load("../../aws_s3/Idrid_224_CN/y_train.npy")
X_test = np.load("../../aws_s3/Idrid_224_CN/X_test.npy")
y_test = np.load("../../aws_s3/Idrid_224_CN/y_test.npy")


X_train = tf.convert_to_tensor(X_train, dtype="float32")
# y_train = tf.convert_to_tensor(y_train, dtype="float32")
X_test = tf.convert_to_tensor(X_test, dtype="float32")
# y_test = tf.convert_to_tensor(y_test, dtype="float32")

In [16]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling="avg",
)

In [17]:
# freesze the base model
base_model.trainable = False
num_classes = 4
epochs = 50
# Define the inputs
inputs = tf.keras.Input(shape=(224, 224, 3))

# Ensure the base_model is running in inference mode.
x = base_model(inputs, training=False)

x = layers.Dropout(0.5)(x)  # Dropout layer with 50% dropout rate
# Adding FC (Fully Connected) layers
x = layers.Dense(1000)(x)
x = layers.Dropout(0.5)(x)  # Another Dropout layer with 50% dropout rate
# Adding FC (Fully Connected) layers
x = layers.Dense(500)(x)

# Adding a final layer with SoftMax activation for classification
outputs = layers.Dense(num_classes, activation="softmax")(x)

# Creating the model
model_cn = tf.keras.Model(inputs=inputs, outputs=outputs)

model_cn.summary(show_trainable=True)

Model: "model_1"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 efficientnetb0 (Functional)  (None, 1280)             4049571   N          
                                                                            
 dropout_2 (Dropout)         (None, 1280)              0         Y          
                                                                            
 dense_3 (Dense)             (None, 1000)              1281000   Y          
                                                                            
 dropout_3 (Dropout)         (None, 1000)              0         Y          
                                                                            
 dense_4 (Dense)             (None, 500)               5005

In [18]:
from tensorflow.keras.optimizers import SGD

# base learning rate
base_learning_rate = 1e-4
# maximum learning rate
max_learning_rate = 1e-2

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9, clipnorm=1.0)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

In [19]:
model_cn.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

print("Fitting the top layer of the model")
model_cn.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Fitting the top layer of the model
Epoch 1/50
16/16 [==============================] - 7s 26ms/step - loss: 1.8780 - accuracy: 0.2053 - lr: 1.0000e-04
Epoch 2/50
16/16 [==============================] - 0s 26ms/step - loss: 1.8274 - accuracy: 0.2337 - lr: 0.0011
Epoch 3/50
16/16 [==============================] - 0s 26ms/step - loss: 1.7367 - accuracy: 0.3598 - lr: 0.0021
Epoch 4/50
16/16 [==============================] - 0s 27ms/step - loss: 1.5013 - accuracy: 0.3699 - lr: 0.0031
Epoch 5/50
16/16 [==============================] - 0s 26ms/step - loss: 1.3388 - accuracy: 0.3882 - lr: 0.0041
Epoch 6/50
16/16 [==============================] - 0s 26ms/step - loss: 1.2427 - accuracy: 0.3821 - lr: 0.0050
Epoch 7/50
16/16 [==============================] - 0s 26ms/step - loss: 1.2158 - accuracy: 0.4573 - lr: 0.0060
Epoch 8/50
16/16 [==============================] - 0s 26ms/step - loss: 1.3139 - accuracy: 0.4614 - lr: 0.0070
Epoch 9/50
16/16 [==============================] - 0s 27ms/step 

In [20]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Unfreeze the base model
base_model.trainable = True
model.summary(show_trainable=True)

# Re-instantiate the optimizer
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9, clipnorm=1.0)

model_cn.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

print("Fitting the end-to-end model")
# Train end-to-end. Stop before overfit
model_cn.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 efficientnetb0 (Functional)  (None, 1280)             4049571   Y          
                                                                            
 dropout (Dropout)           (None, 1280)              0         Y          
                                                                            
 dense (Dense)               (None, 1000)              1281000   Y          
                                                                            
 dropout_1 (Dropout)         (None, 1000)              0         Y          
                                                                            
 dense_1 (Dense)             (None, 500)               500500

In [21]:
print("Test dataset evaluation color normalization")
model_cn.evaluate(X_test, y_test)

Test dataset evaluation color normalization
1/1 [==============================] - 2s 2s/step - loss: 3.9113 - accuracy: 0.5000


[3.9112510681152344, 0.5]

In [22]:
# 20 epochs, tensor X 0.5000
# 20 epochs, tensor X 0.5000
# 20 epochs, tensor X 0.5833
# 20 epochs, tensor X 0.5000
# 20 epochs, tensor X 0.3750
# 20 epochs, tensor X 0.5000
# 20 epochs, tensor X 0.5000

In [23]:
# 50 epochs, tensor X,  0.41
# 50 epochs, tensor X,  0.50